In [153]:
# https://www.geeksforgeeks.org/sparse-matrix-representations-set-3-csr/
struct MyCSR
    m::Int64                  # Number of rows
    n::Int64                  # Number of columns
    rowptr::Vector{Int64}     # Row j is in rowptr[j]:(rowptr[j+1]-1)
    colval::Vector{Int64}     # Col indices of stored values
    nzval::Vector{Float64}    # Stored values, typically nonzeros
    function MyCSR(m::Int64, n::Int64)
        rowptr=zeros(Int64, m+1)
        colval=Vector{Int64}()
        nzval=Vector{Float64}()
        return new(m, n,rowptr,colval,nzval)
    end
    MyCSR(m,n,rowptr,colval,nzval)=new(m,n,rowptr,colval, nzval)
end
# CSR is good! remember me!!!!!!
# Base.getindex(::MyCSR, i::Int64, j::Int64) = 3



In [154]:
import Base.*
function *(M::MyCSR, v::Vector{Float64})
    nw=zeros(length(v))
    for row in 1:M.m
        for i in M.rowptr[row]:M.rowptr[row+1]-1
            col=M.colval[i]
            nw[row]+=v[col]*M.nzval[i]
        end
    end
    return nw
end 

* (generic function with 329 methods)

In [155]:
function Densify(a::MyCSR)
    M=zeros(Float64, a.m, a.n)
    for row in 1:a.m
        for i in a.rowptr[row]:a.rowptr[row+1]-1
            c=a.colval[i]
            M[row, c]=a.nzval[i]
        end
    end
    return M
end

Densify (generic function with 1 method)

In [156]:
using LinearAlgebra;
using Latexify;

In [157]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"])

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [158]:
input=dummy

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [168]:
damping=0.85
function get_matrix(exitLinks)
    n=length(dummy)
    exitCount=Dict{Int64, Int64}()
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, list) in exitLinks
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        push!(exitCount, i=>length(list))
        i+=1
    end
    
    
    enter=Vector{Vector{Int64}}()
    for _ in 1:n
        push!(enter, Vector{Int64}())
    end
    for (f, list) in exitLinks 
        from=decodeNames[f]
        for t in list 
            to=decodeNames[t]
            push!(enter[to], from)
        end
    end
    
    
    rowptr=ones(Int64, n+1)
    colval=Vector{Int64}()
    nzval=Vector{Float64}()
    i=2
    for to in 1:length(enter) 
        rowptr[i]=rowptr[i-1]+length(enter[to])
        for from in enter[to]
            append!(nzval, 1/exitCount[from])
            append!(colval, from)
        end
        i+=1
    end
    return (MyCSR(n,n, rowptr, colval, nzval), decodeNames, encodeNa, n)
end

get_matrix (generic function with 1 method)

In [169]:
function iterative(input, k)
    M, encode, decode, n = get_matrix(input)
    v = ones(n)/n
    for i in 1:k
        v=M*v
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by= x->x[2], rev=true)
end

iterative (generic function with 1 method)

In [171]:
M, encode, decode, n = get_matrix(input)

(MyCSR(4, 4, [1, 2, 4, 6, 7], [3, 3, 4, 1, 2, 1], [0.5, 0.5, 1.0, 0.5, 1.0, 0.5]), Dict("c" => 1, "b" => 2, "a" => 3, "d" => 4), [[3], [3, 4], [1, 2], [1]], 4)

In [176]:
v = ones(n)/n
for i in 1:40
    v=M*v
end
v

4-element Vector{Float64}:
 0.20000000000004547
 0.2999999999999545
 0.3999999999998636
 0.10000000000013642

In [170]:
iterative(input, 40)

LoadError: KeyError: key 1 not found